In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
chat = ChatOpenAI(temperature=0.1)

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [16]:
class CommaOutputParser(BaseOutputParser):
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [38]:
template = ChatPromptTemplate.from_messages([
    ("system","당신은 list 생성 기계입니다. 입력받은 질문들은 모두 콤마로 구분된된 list로 답해질 것입니다. 최대 {max_items}개 만큼이요. list가 아닌 것으로는 답을 하지 마세요."),
    ("human","{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="행성은 어떤 것들이 있나요?"
)

result = chat.predict_messages(prompt)

In [41]:
p.parse(result.content)

['수성', '금성', '지구', '화성', '목성', '토성', '천왕성', '해왕성', '명왕성']

In [42]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"포켓몬을 알려줘."
})

['피카츄', '파이리', '꼬부기', '이상해씨', '꼬북이']